In [ ]:
!pip install yfinance
!pip install yahoofinancials

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.2 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for yahoofinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15191 sha256=b33ec9313cd5be4f170a50f356d322c1cf290562d5f114e2288f32f4cbfc4d59
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built yahoofinancials


In [ ]:
import numpy as np
import time as tm
import datetime 
import tensorflow as tf
import pandas as pd
# Data preparation
from sklearn.preprocessing import MinMaxScaler
from collections import deque

# AI
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

# Graphics library
import matplotlib.pyplot as plt

import yfinance as yf
from yahoofinancials import YahooFinancials
import time
import datetime
import os
from google.colab import files

In [ ]:
#time to pull the data
start_date =datetime.datetime.today().strftime('%Y-%m-%d')
end_date = (datetime.datetime.today() - datetime.timedelta(days=1100)).strftime('%Y-%m-%d')

In [ ]:
# calling API 
data=YahooFinancials('SOL-USD').get_historical_price_data(end_date,start_date,"daily")
#into the dataframe
sol_df = pd.DataFrame(data['SOL-USD']['prices'])

sol_df =sol_df.drop(['high','low','open','volume','date','adjclose'], axis=1)
sol_df.rename(columns={'formatted_date':'date'},inplace=True)
sol_df['date']=pd.to_datetime(sol_df ['date'])


In [ ]:
sol_df

,close,date
0,0.951054,2020-04-10
1,0.776819,2020-04-11
2,0.882507,2020-04-12
3,0.777832,2020-04-13
4,0.661925,2020-04-14
...,...,...
941,29.601345,2022-11-07
942,24.152937,2022-11-08
943,13.940857,2022-11-09
944,17.681408,2022-11-10


In [ ]:
# SETTINGS

# Window size or the sequence length, 7 (1 week)
N_STEPS = 7

# Lookup steps, 1 is the next day, 3 = after tomorrow
LOOKUP_STEPS = [1, 2, 3]

In [ ]:
sol_df.dtypes

close           float64
date     datetime64[ns]
dtype: object

In [ ]:
# Scale data for ML engine
scaler = MinMaxScaler()
sol_df['scaled_close'] = scaler.fit_transform(np.expand_dims(sol_df['close'].values, axis=1))


In [ ]:
def PrepareData(days):
    df = sol_df.copy()
    df['future'] = df['scaled_close'].shift(-days)
    last_sequence = np.array(df[['scaled_close']].tail(days))
    df.dropna(inplace=True)
    sequence_data = []
    sequences = deque(maxlen=N_STEPS)
    
    for entry, target in zip(df[['scaled_close'] + ['date']].values, df['future'].values):
        sequences.append(entry)
        if len(sequences) == N_STEPS:
            sequence_data.append([np.array(sequences), target])
            
    last_sequence = list([s[:len(['scaled_close'])] for s in sequences]) + list(last_sequence)
    last_sequence = np.array(last_sequence).astype(np.float32)

    # construct the X's and Y's
    X, Y = [], []
    for seq, target in sequence_data:
        X.append(seq)
        Y.append(target)

   # convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)

    return df, last_sequence, X, Y

In [ ]:
PrepareData(3) # 3 days


(         close       date  scaled_close    future
 0     0.951054 2020-04-10      0.001686  0.001016
 1     0.776819 2020-04-11      0.001012  0.000567
 2     0.882507 2020-04-12      0.001421  0.000508
 3     0.777832 2020-04-13      0.001016  0.000679
 4     0.661925 2020-04-14      0.000567  0.000563
 ..         ...        ...           ...       ...
 938  33.780853 2022-11-04      0.128727  0.112554
 939  36.765762 2022-11-05      0.140278  0.091470
 940  32.683582 2022-11-06      0.124481  0.051953
 941  29.601345 2022-11-07      0.112554  0.066428
 942  24.152937 2022-11-08      0.091470  0.063385
 
 [943 rows x 4 columns], array([[0.11714617],
        [0.11736211],
        [0.12872727],
        [0.14027792],
        [0.12448118],
        [0.1125539 ],
        [0.09147029],
        [0.05195276],
        [0.06642751],
        [0.06338514]], dtype=float32), array([[[0.0016863346438802506, Timestamp('2020-04-10 00:00:00')],
         [0.0010121003560064562, Timestamp('2020-04-11 00:

In [ ]:
def GetTrainedModel(x_train, y_train):
  model = Sequential()
  model.add(LSTM(60, return_sequences=True, input_shape=(N_STEPS, len(['scaled_close']))))
  model.add(Dropout(0.3))
  model.add(LSTM(120, return_sequences=False))
  model.add(Dropout(0.3))
  model.add(Dense(20))
  model.add(Dense(1))

  BATCH_SIZE = 8
  EPOCHS = 80

  model.compile(loss='mean_squared_error', optimizer='adam')

  model.fit(x_train, y_train,
            batch_size=BATCH_SIZE,
            epochs=EPOCHS,
            verbose=1)

  model.summary()

  return model

In [ ]:
# GET PREDICTIONS
predictions = []

for step in LOOKUP_STEPS:
  df, last_sequence, x_train, y_train = PrepareData(step)
  x_train = x_train[:, :, :len(['scaled_close'])].astype(np.float32)

  model = GetTrainedModel(x_train, y_train)

  last_sequence = last_sequence[-N_STEPS:]
  last_sequence = np.expand_dims(last_sequence, axis=0)
  prediction = model.predict(last_sequence)
  predicted_price = scaler.inverse_transform(prediction)[0][0]

  predictions.append(round(float(predicted_price), 2))

Epoch 1/80
118/118 [==============================] - 7s 14ms/step - loss: 0.0075
Epoch 2/80
118/118 [==============================] - 2s 14ms/step - loss: 0.0021
Epoch 3/80
118/118 [==============================] - 3s 25ms/step - loss: 0.0028
Epoch 4/80
118/118 [==============================] - 3s 23ms/step - loss: 0.0018
Epoch 5/80
118/118 [==============================] - 3s 25ms/step - loss: 0.0021
Epoch 6/80
118/118 [==============================] - 3s 25ms/step - loss: 0.0022
Epoch 7/80
118/118 [==============================] - 3s 25ms/step - loss: 0.0019
Epoch 8/80
118/118 [==============================] - 3s 26ms/step - loss: 0.0018
Epoch 9/80
118/118 [==============================] - 3s 25ms/step - loss: 0.0018
Epoch 10/80
118/118 [==============================] - 3s 24ms/step - loss: 0.0020
Epoch 11/80
118/118 [==============================] - 3s 22ms/step - loss: 0.0016
Epoch 12/80
118/118 [==============================] - 2s 14ms/step - loss: 0.0016
Epoch 13/80
1

In [ ]:
if bool(predictions) == True and len(predictions) > 0:
  predictions_list = [str(d)+'$' for d in predictions]
  predictions_str = ', '.join(predictions_list)
  message = f' prediction for upcoming 3 days ({predictions_str})'
  
  print(message)

 prediction for upcoming 3 days (16.7$, 15.75$, 15.57$)


In [15]:
from google.colab import files
today=start_date =datetime.datetime.today()
day1=(today+datetime.timedelta(days=1)).strftime('%Y-%m-%d')
day2=(today+datetime.timedelta(days=2)).strftime('%Y-%m-%d')
day3=(today+datetime.timedelta(days=3)).strftime('%Y-%m-%d')
days=[day1,day2,day3]
sol_lstm=pd.DataFrame()
sol_lstm['future days']=days
sol_lstm['price']=predictions
sol_lstm['ticker']='SOL'

sol_lstm.to_csv('sol_lstm.csv') 
files.download('sol_lstm.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>